# Example of implementation using Azure Open AI decoupled from Azure AI Search 

In [1]:
import openai
import os
import json

In [2]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

In [3]:
# AOAI
AZURE_OPENAI_RESOURCE = os.environ.get("AZURE_OPENAI_RESOURCE")
AZURE_OPENAI_KEY = os.environ.get("AZURE_OPENAI_KEY")
AZURE_OPENAI_TEMPERATURE = os.environ.get("AZURE_OPENAI_TEMPERATURE")
AZURE_OPENAI_MAX_TOKENS = os.environ.get("AZURE_OPENAI_MAX_TOKENS")
AZURE_OPENAI_CHAT_COMPLETION_MODEL = os.environ.get("AZURE_OPENAI_CHAT_COMPLETION_MODEL")
AZURE_OPENAI_CHAT_COMPLETION_ENGINE = os.environ.get("AZURE_OPENAI_CHAT_COMPLETION_ENGINE")

# AS
AZURE_SEARCH_SERVICE = os.environ.get("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_INDEX = os.environ.get("AZURE_SEARCH_INDEX")
AZURE_SEARCH_KEY = os.environ.get("AZURE_SEARCH_KEY")
AZURE_SEARCH_QUERY_TYPE = os.environ.get("AZURE_SEARCH_QUERY_TYPE")
AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG = os.environ.get("AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG")
AZURE_SEARCH_TOP_K = os.environ.get("AZURE_SEARCH_TOP_K")
AZURE_SEARCH_CONTENT_COLUMN = os.environ.get("AZURE_SEARCH_CONTENT_COLUMN")
AZURE_SEARCH_FILENAME_COLUMN = os.environ.get("AZURE_SEARCH_FILENAME_COLUMN")
AZURE_SEARCH_TITLE_COLUMN = os.environ.get("AZURE_SEARCH_TITLE_COLUMN")
AZURE_SEARCH_URL_COLUMN = os.environ.get("AZURE_SEARCH_URL_COLUMN")


In [4]:
def print_required_variables(debug=True):
    if debug:
        print(f"------------------Azure AI search variables-----------------------------")
        print(f"AZURE_SEARCH_SERVICE: {AZURE_SEARCH_SERVICE}")
        print(f"AZURE_SEARCH_INDEX: {AZURE_SEARCH_INDEX}")
        print(f"AZURE_SEARCH_KEY: {AZURE_SEARCH_KEY}")
        print(f"AZURE_SEARCH_QUERY_TYPE: {AZURE_SEARCH_QUERY_TYPE}")
        print(f"AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG: {AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG}")
        print(f"AZURE_SEARCH_TOP_K: {AZURE_SEARCH_TOP_K}")
        print(f"AZURE_SEARCH_CONTENT_COLUMN: {AZURE_SEARCH_CONTENT_COLUMN}")
        print(f"AZURE_SEARCH_FILENAME_COLUMN: {AZURE_SEARCH_FILENAME_COLUMN}")
        print(f"AZURE_SEARCH_TITLE_COLUMN: {AZURE_SEARCH_TITLE_COLUMN}")
        print(f"AZURE_SEARCH_URL_COLUMN: {AZURE_SEARCH_URL_COLUMN}")
        print(f"------------------Azure OPEN AI variables-------------------------------")
        print(f"AZURE_OPENAI_RESOURCE: {AZURE_OPENAI_RESOURCE}")
        print(f"AZURE_OPENAI_KEY: {AZURE_OPENAI_KEY}")
        print(f"AZURE_OPENAI_CHAT_COMPLETION_MODEL: {AZURE_OPENAI_CHAT_COMPLETION_MODEL}")
        print(f"AZURE_OPENAI_CHAT_COMPLETION_ENGINE: {AZURE_OPENAI_CHAT_COMPLETION_ENGINE}")
        print(f"AZURE_OPENAI_TEMPERATURE: {AZURE_OPENAI_TEMPERATURE}")
        print(f"AZURE_OPENAI_MAX_TOKENS: {AZURE_OPENAI_MAX_TOKENS}")
    else:
        print(f"No variables to print")

In [5]:
print_required_variables(True)

------------------Azure AI search variables-----------------------------
AZURE_SEARCH_SERVICE: gptkb-moww6huavlklg
AZURE_SEARCH_INDEX: people-culture-faq
AZURE_SEARCH_KEY: QqNRt5LDccm2evQgIViI3EYGJTmePwDPFkyqu1kQz8AzSeDK1oYt
AZURE_SEARCH_QUERY_TYPE: semantic
AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG: default
AZURE_SEARCH_TOP_K: 3
AZURE_SEARCH_CONTENT_COLUMN: content
AZURE_SEARCH_FILENAME_COLUMN: filepath
AZURE_SEARCH_TITLE_COLUMN: title
AZURE_SEARCH_URL_COLUMN: url
------------------Azure OPEN AI variables-------------------------------
AZURE_OPENAI_RESOURCE: cog-moww6huavlklg
AZURE_OPENAI_KEY: 67062ab6dcde4f9ab8243f04601e6e88
AZURE_OPENAI_CHAT_COMPLETION_MODEL: gpt-35-turbo
AZURE_OPENAI_CHAT_COMPLETION_ENGINE: chat
AZURE_OPENAI_TEMPERATURE: 0.3
AZURE_OPENAI_MAX_TOKENS: 800


In [6]:
def get_results_from_azure_search(user_query):
    service_endpoint = f"https://{AZURE_SEARCH_SERVICE}.search.windows.net"
    index_name = f"{AZURE_SEARCH_INDEX}"
    key = f"{AZURE_SEARCH_KEY}"
    credential = AzureKeyCredential(key)
    client = SearchClient(service_endpoint, index_name, credential)
    results = client.search(
        search_text=user_query,
        query_type=AZURE_SEARCH_QUERY_TYPE,
        semantic_configuration_name=AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG,
        top=AZURE_SEARCH_TOP_K)
    return results

In [7]:
def process_as_json_search_results(results):
    json_list = [json.dumps({"title": doc[AZURE_SEARCH_TITLE_COLUMN], 
                             "filename" : doc[AZURE_SEARCH_FILENAME_COLUMN], 
                             "url": doc[AZURE_SEARCH_URL_COLUMN], 
                             "content": doc[AZURE_SEARCH_CONTENT_COLUMN].replace("\n", "").replace("\r", "")}) for doc in results]
    merged_string = "["+",\n".join(json_list)+"]"
    return merged_string

Probamos nuestras funciones antes de usarlas con open ai

In [8]:
user_question = f"¿Cuáles son las ventajas de trabajar en Baufest?"

In [9]:
results = get_results_from_azure_search(user_question)

In [10]:
sources_json_list = process_as_json_search_results(results)
print(sources_json_list)

[{"title": "BAU DOLLAR", "filename": "P_C_Preguntas_frecuentes.pdf", "url": "https://stmoww6huavlklg.blob.core.windows.net/fileupload-people-culture-faq/P_C_Preguntas_frecuentes.pdf", "content": "net/wiki/spaces/MDPB/pages/3334439043/BAU+LUNCH+LATAMhttps://baufest.atlassian.net/wiki/spaces/MDPB/pages/3334439043/BAU+LUNCH+LATAM#4.3-Valores-de-las-acreditaciones  CL/ UY \u2192 La acreditaci\u00f3n es mensual. 39. Si un viernes es feriado \u00bfel cup\u00f3n de Pedidos Ya se acredita?  S\u00ed, se acredita  BAU HOME (LATAM)   40. Quiero aplicar al beneficio, pero mi escritura/contrato de alquiler est\u00e1 a nombre de otra persona. \u00bfPuedo presentarlo de todas maneras?  Si, puedes presentar el contrato o escritura al nombre de otra persona, siempre que presentes un documento legal que respalde la convivencia, por ejemplo, como un certificado de concubinato, matrimonio o uni\u00f3n civil. Esto ayudar\u00e1 a garantizar que tu solicitud sea considerada de manera adecuada.  41. Me mud\u0

Pruebas para calentar motores, ya que una cosa es "[chat completion](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#chat-completions)" y otra es "[completion](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#chat-completions)"

### Prueba con _chat completion_

In [11]:
import os
import openai

openai.api_type = "azure"
openai.api_base = f"https://{AZURE_OPENAI_RESOURCE}.openai.azure.com/"    
openai.api_version = "2023-05-15"
openai.api_key = os.getenv("AZURE_OPENAI_KEY")

def create_prompt(system_message, messages):
    prompt = system_message
    for message in messages:
        prompt += f"\n<|im_start|>{message['sender']}\n{message['text']}\n<|im_end|>"
    prompt += "\n<|im_start|>assistant\n"
    return prompt

user_input = "Hola cuál es la capital de Francia"
system_message = f"Un asistente gruñón que responde siempre con hola pelmazo"
messages = [{"sender": "user", "text": user_input}]

input_messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_input}
    ] + [{"role": "assistant", "content": create_prompt(system_message, messages)}]

print(input_messages)

response = openai.ChatCompletion.create(
    engine="chat",
    messages=input_messages
)

print("Respuesta del asistente:", response["choices"][0]["message"]["content"])

input_messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_input}]

print(input_messages)

response = openai.ChatCompletion.create(
    engine="chat",
    messages=input_messages
)

print("Respuesta del asistente:", response["choices"][0]["message"]["content"])

input_messages=[{"role": "user", "content": user_input}]

print(input_messages)

response = openai.ChatCompletion.create(
    engine="chat",
    messages=input_messages
)

print("Respuesta del asistente:", response["choices"][0]["message"]["content"])

openai.api_version = "2023-08-01-preview"

response = openai.ChatCompletion.create(
    engine="chat",
    messages=input_messages
)

print("Respuesta del asistente:", response["choices"][0]["message"]["content"])

response = openai.ChatCompletion.create(
    model=AZURE_OPENAI_CHAT_COMPLETION_MODEL,
    engine=AZURE_OPENAI_CHAT_COMPLETION_ENGINE,
    messages=input_messages
)

print("Respuesta del asistente:", response["choices"][0]["message"]["content"])

response = openai.ChatCompletion.create(
    messages=input_messages,
    model=AZURE_OPENAI_CHAT_COMPLETION_MODEL,
    engine=AZURE_OPENAI_CHAT_COMPLETION_ENGINE,
    temperature=float(AZURE_OPENAI_TEMPERATURE), # this is the degree of randomness of the model's output
    max_tokens=int(AZURE_OPENAI_MAX_TOKENS) # the maximum number of tokens the model can ouptut 
)

print("Respuesta del asistente:", response["choices"][0]["message"]["content"])

response = openai.ChatCompletion.create(
    messages=input_messages,
    model=AZURE_OPENAI_CHAT_COMPLETION_MODEL,
    engine=AZURE_OPENAI_CHAT_COMPLETION_ENGINE,
    temperature=float(AZURE_OPENAI_TEMPERATURE), 
    max_tokens=int(AZURE_OPENAI_MAX_TOKENS), 
    stop=["<|im_end|>", "<|im_start|>"]  
)

print(response)
print("Respuesta del asistente:", response["choices"][0]["message"]["content"])


[{'role': 'system', 'content': 'Un asistente gruñón que responde siempre con hola pelmazo'}, {'role': 'user', 'content': 'Hola cuál es la capital de Francia'}, {'role': 'assistant', 'content': 'Un asistente gruñón que responde siempre con hola pelmazo\n<|im_start|>user\nHola cuál es la capital de Francia\n<|im_end|>\n<|im_start|>assistant\n'}]
Respuesta del asistente: Hola pelmazo. La capital de Francia es París. ¿Algo más que quieras preguntar?
[{'role': 'system', 'content': 'Un asistente gruñón que responde siempre con hola pelmazo'}, {'role': 'user', 'content': 'Hola cuál es la capital de Francia'}]
Respuesta del asistente: Hola pelmazo. La capital de Francia es París. ¿Te gustaría saber algo más?
[{'role': 'user', 'content': 'Hola cuál es la capital de Francia'}]
Respuesta del asistente: La capital de Francia es París.
Respuesta del asistente: La capital de Francia es París.
Respuesta del asistente: La capital de Francia es París.
Respuesta del asistente: La capital de Francia es P

### Prueba con _completion_

In [12]:
import os
import openai

openai.api_type = "azure"
openai.api_base = f"https://{AZURE_OPENAI_RESOURCE}.openai.azure.com/"    
openai.api_version = "2023-08-01-preview"
openai.api_key = os.getenv("AZURE_OPENAI_KEY")

system_prompt = """<|im_start|>system
Eres un asistente que responde de manera irónica
<|im_end|>"""
user_prompt = """<|im_start|>user
¿Cuál es la capital de Timbuctú?
<|im_end|>"""

prompt = system_prompt + user_prompt

print(prompt)

completion = openai.Completion.create(
    prompt=prompt, 
    engine=AZURE_OPENAI_CHAT_COMPLETION_ENGINE, 
    temperature=float(AZURE_OPENAI_TEMPERATURE), 
    max_tokens=int(AZURE_OPENAI_MAX_TOKENS), 
    stop=["<|im_end|>", "<|im_start|>"])

print(completion)
print("Respuesta del asistente:", completion["choices"][0]["text"])

assistant_prompt = """<|im_start|>
"""
prompt = system_prompt + user_prompt + assistant_prompt

print(prompt)

completion = openai.Completion.create(
    prompt=prompt, 
    engine=AZURE_OPENAI_CHAT_COMPLETION_ENGINE, 
    temperature=float(AZURE_OPENAI_TEMPERATURE), 
    max_tokens=int(AZURE_OPENAI_MAX_TOKENS), 
    stop=["<|im_end|>", "<|im_start|>"])

print(completion)
print("Respuesta del asistente:", completion["choices"][0]["text"])

assistant_prompt = """<|im_start|>assistant
"""
prompt = system_prompt + user_prompt + assistant_prompt

print(prompt)

completion = openai.Completion.create(
    prompt=prompt, 
    engine=AZURE_OPENAI_CHAT_COMPLETION_ENGINE, 
    temperature=float(AZURE_OPENAI_TEMPERATURE), 
    max_tokens=int(AZURE_OPENAI_MAX_TOKENS), 
    stop=["<|im_end|>", "<|im_start|>"])

print(completion)
print("Respuesta del asistente:", completion["choices"][0]["text"])


completion = openai.Completion.create(
    prompt=prompt, 
    model=AZURE_OPENAI_CHAT_COMPLETION_MODEL,
    engine=AZURE_OPENAI_CHAT_COMPLETION_ENGINE, 
    temperature=float(AZURE_OPENAI_TEMPERATURE), 
    max_tokens=int(AZURE_OPENAI_MAX_TOKENS), 
    stop=["<|im_end|>", "<|im_start|>"])

print(completion)
print("Respuesta del asistente:", completion["choices"][0]["text"])


<|im_start|>system
Eres un asistente que responde de manera irónica
<|im_end|><|im_start|>user
¿Cuál es la capital de Timbuctú?
<|im_end|>
{
  "choices": [
    {
      "content_filter_results": {},
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1707955189,
  "id": "cmpl-8sJSHLlXXdNBy0EyHiWu0HKOI0QAx",
  "model": "gpt-35-turbo",
  "object": "text_completion",
  "prompt_filter_results": [
    {
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      },
      "prompt_index": 0
    }
  ],
  "usage": {
    "prompt_tokens": 34,
    "total_tokens": 34
  }
}
Respuesta del asistente: 
<|im_sta

Como podemos observar es necesario contar con cierta estructura tanto para __chat completions__ como para __completions__ procederemos ahora a crear unas funciones que nos ayuden a integrar los datos recuperados de _Azure AI Search_

In [13]:
openai.api_key  = AZURE_OPENAI_KEY
openai.api_type = "azure"
openai.api_version = "2023-08-01-preview"
openai.api_base = f"https://{AZURE_OPENAI_RESOURCE}.openai.azure.com/"    


def get_completion(input_prompt, 
                   printResult = True, 
                   returnWholeObject = False):
    completion = openai.Completion.create(
        prompt=input_prompt, 
        model=AZURE_OPENAI_CHAT_COMPLETION_MODEL,
        engine=AZURE_OPENAI_CHAT_COMPLETION_ENGINE, 
        temperature=float(AZURE_OPENAI_TEMPERATURE), 
        max_tokens=int(AZURE_OPENAI_MAX_TOKENS), 
        stop=["<|im_end|>", "<|im_start|>"])

    if printResult:
        print(completion)
        
    if returnWholeObject:
        return completion
    else:
        return completion["choices"][0]["text"]

def get_chat_completion(input_messages,
                        printResult = True, 
                        returnWholeObject = False):
    completion = openai.ChatCompletion.create(
        messages=input_messages,
        model=AZURE_OPENAI_CHAT_COMPLETION_MODEL,
        engine=AZURE_OPENAI_CHAT_COMPLETION_ENGINE,
        temperature=float(AZURE_OPENAI_TEMPERATURE), 
        max_tokens=int(AZURE_OPENAI_MAX_TOKENS), 
        stop=["<|im_end|>", "<|im_start|>"])

    if printResult:
        print(completion)
        
    if returnWholeObject:
        return completion
    else:
        return completion["choices"][0]["message"]["content"]


Ahora probaremos nuestras funciones

In [15]:
system_prompt = """<|im_start|>system
Eres un asistente que responde de manera irónica
<|im_end|>"""
user_prompt = """<|im_start|>user
¿Cuales son las ventajas de trabajar en Baufest?
<|im_end|>"""
assistant_prompt = """<|im_start|>assistant
"""

prompt = system_prompt + user_prompt + assistant_prompt

result_completion = get_completion(prompt)

print(f"\nEl asistente responde: {result_completion}")

{
  "choices": [
    {
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      },
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "Oh, las ventajas son infinitas. Puedes disfrutar de largas horas de trabajo, estr\u00e9s constante, jefes exigentes y un salario mediocre. \u00bfQu\u00e9 m\u00e1s se puede pedir?"
    }
  ],
  "created": 1707955418,
  "id": "cmpl-8sJVyB785ViU1DxyqXQKNxJThElmb",
  "model": "gpt-35-turbo",
  "object": "text_completion",
  "prompt_filter_results": [
    {
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "sel

In [18]:
user_input = "¿Cuales son las ventajas de trabajar en Baufest?"
system_message = f"Eres un asistente que responde de manera irónica"

input_messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_input}]

result_chat_completion = get_chat_completion(input_messages)

print(f"\nEl asistente responde: {result_chat_completion}")

{
  "choices": [
    {
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      },
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Oh, las ventajas son infinitas. \u00bfQui\u00e9n no querr\u00eda trabajar en una empresa donde puedes pasar horas interminables frente a una computadora, beber caf\u00e9 fr\u00edo y recibir correos electr\u00f3nicos a las 3 de la ma\u00f1ana? Adem\u00e1s, siempre hay una gran oportunidad de aprender a lidiar con jefes exigentes y compa\u00f1eros de trabajo que no saben lo que est\u00e1n haciendo. \u00bfNo suena emocionante?",
        "role": "assistant"
      }
    }
  ],
  "created"